## Import Needed Libraries

In [ ]:
from  texas_gerrymandering_hb4.config import FINAL_CSV
import json
import os
import joblib

from sklearn.preprocessing import StandardScaler

In [ ]:
RACE_COLS = ["pct_white", "pct_black", "pct_asian", "pct_hispanic"]
ID_COL = "district_id"

In [ ]:
def build_features(df, drop_race: bool):
    cols = [c for c in df.columns if c != ID_COL]
    if drop_race:
        cols = [c for c in cols if c not in RACE_COLS]
    return df[cols].copy(), cols

# Full set
X_full, full_cols = build_features(df, drop_race=False)
scaler_full = StandardScaler()
X_full_scaled = scaler_full.fit_transform(X_full)

# No-race set
X_norace, norace_cols = build_features(df, drop_race=True)
scaler_norace = StandardScaler()
X_norace_scaled = scaler_norace.fit_transform(X_norace)

In [ ]:
# Save artifacts
joblib.dump(scaler_full, f"{ART_DIR}/scaler_full.joblib")
np.savez(f"{ART_DIR}/X_full_scaled.npz", X=X_full_scaled)
with open(f"{ART_DIR}/full_columns.json", "w") as f: json.dump(full_cols, f)

joblib.dump(scaler_norace, f"{ART_DIR}/scaler_norace.joblib")
np.savez(f"{ART_DIR}/X_norace_scaled.npz", X=X_norace_scaled)
with open(f"{ART_DIR}/norace_columns.json", "w") as f: json.dump(norace_cols, f)

df[[ID_COL]].to_csv(f"{ART_DIR}/district_ids.csv", index=False)
df.to_csv(f"{ART_DIR}/dataset_snapshot.csv", index=False)
print("Artifacts saved in", ART_DIR)